In [33]:
import pandas as pd

In [34]:
li = []
for i in range(9):
    temp = pd.read_csv(f"GSE_{2014 + i}.csv")
    li.append(temp)
    
df = pd.concat(li)
df.head()

,service_date,time_period,stop_id,station_name,route_or_line,gated_entries
0,2014-12-31,(00:00:00),place-welln,Wellington,Orange Line,1.00
1,2014-12-31,(00:30:00),place-tumnl,Tufts Medical Center,Orange Line,1.00
2,2014-12-31,(02:00:00),place-sdmnl,Suffolk Downs,Blue Line,1.00
3,2014-12-31,(03:00:00),place-asmnl,Ashmont,Red Line,1.00
4,2014-12-31,(03:00:00),place-dwnxg,Downtown Crossing,Orange Line,0.51


In [35]:
df.dtypes

service_date      object
time_period       object
stop_id           object
station_name      object
route_or_line     object
gated_entries    float64
dtype: object

In [36]:
df = df.groupby(by=["service_date", "time_period", "route_or_line"]).sum()
df.head()

gated_entries
service_date time_period route_or_line               
2014-01-01   (00:00:00)  Blue Line             113.70
                         Green Line            197.25
                         Orange Line           208.53
                         Red Line              339.12
                         Silver Line            25.40

In [29]:
df.to_csv('mbta_clean.csv')

In [39]:
df.sample(frac=0.1).to_csv('mbta_clean_small.csv')

In [42]:
df.sample(frac=0.01).to_csv('mbta_clean_micro.csv')